In [1]:
import os,sys
import numpy as np
import scipy as sp
import pandas as pd
#from netCDF4 import Dataset
import matplotlib
from datetime import datetime, timedelta, time
import glob
from generate_atmosphere import LidarProfile,RadiosondeProfile
import miscLidar as mscLid
from molecular import rayleigh_scattering
import global_settings as gs
import pandas as pd
#import ARLreader as Ar
import preprocessing as prep
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
%matplotlib inline


C:\Users\addalin\Dropbox\Lidar\code\miscLidar.py:40: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  plt.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}"]


In [2]:
'''set day'''
day_date = datetime(2017,9,1)

'''set station'''
haifa_station = gs.station()

'''Set paths'''
#lidar_parent_folder = 'H:\data_haifa\DATA FROM TROPOS\data\level1a\PollyXT_TROPOS'
#gdas_parent_folder = 'H:\data_haifa\DATA FROM TROPOS\GDAS\haifa'
lidar_parent_folder = haifa_station.lidar_src_folder
haifa_station.gdas1_folder = os.path.join('.','data examples','gdas')
haifa_station.gdastxt_folder  = os.path.join('.','data examples','gdas_txt')
gdas1_folder,gdas1_paths = prep.get_gdas_paths(haifa_station, day_date,'gdas1')

gdastxt_folder, gdastxt_paths = prep.get_gdas_paths(haifa_station, day_date,'txt')

print(gdastxt_paths)

[]


In [18]:
gdastxt_new =[]
gdastxt_new = prep.convert_daily_gdas ( day_date , haifa_station )

In [19]:
next_day = day_date + timedelta ( days = 1 )
gdastxt_new.extend(prep.convert_daily_gdas ( next_day , haifa_station ))
gdastxt_new

['.\\data examples\\gdas_txt\\2017\\09\\haifa_20170901_00_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170901_03_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170901_06_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170901_09_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170901_12_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170901_15_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170901_18_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170901_21_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170902_00_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170902_03_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170902_06_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170902_09_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170902_12_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09

['.\\data examples\\gdas_txt\\2017\\09\\haifa_20170902_00_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170902_03_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170902_06_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170902_09_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170902_12_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170902_15_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170902_18_32.8_35.0.txt',
 '.\\data examples\\gdas_txt\\2017\\09\\haifa_20170902_21_32.8_35.0.txt']

In [ ]:
bsc_paths, profile_paths = prep.load_att_bsc(lidar_parent_folder, day_date)
bsc_paths
profile_paths

In [5]:
haifa_station = gs.station()
#print(haifa_station.__dict__)
src_folder = os.path.join(os.getcwd(),'data examples\gdas')
dst_folder = os.path.join(os.getcwd(),'data examples\gdas_txt')
gdas_dst_paths = prep.daily_gdas2txt(day_date, haifa_station.location,
                                   haifa_station.lat, haifa_station.lon,
                                   src_folder, dst_folder)

In [ ]:
'''set parameters'''
lambda_um = gs.LAMBDA_um().G
haifa_station = gs.station()
min_height_km = (1E-3)*(haifa_station.altitude + haifa_station.start_bin_height)
top_height_km = (1E-3)*(haifa_station.altitude + haifa_station.end_bin_height)

df_sigma, df_beta = prep.generate_daily_molecular_profile(
	gdas_dst_paths,lambda_um,haifa_station.location, haifa_station.lat,
	haifa_station.lon,min_height_km , top_height_km,haifa_station.n_bins)

#df_sigma

In [1]:
plt.figure()
df_beta.plot()
plt.show()

NameError: name 'plt' is not defined

In [19]:
def generate_daily_att_bsc_mol(lambda_um, station, time_res='30S'):
	"""	 """

	'''Load daily gdas profiles and convert to backscatter (beta) and extinction (sigma) profiles'''
	min_height_km = (station.altitude + station.start_bin_height) * 1E-3
	top_height_km = (station.altitude + station.end_bin_height) * 1E-3

	df_sigma, df_beta = prep.generate_daily_molecular_profile(
		gdas_dst_paths,lambda_um,station.location, station.lat,
		station.lon,min_height_km , top_height_km,station.n_bins)

	''' Interpolate profiles through 24 hrs'''
	interp_sigma_df=(df_sigma.T.resample(time_res).interpolate(method='linear')[:-1]).T
	interp_beta_df=(df_beta.T.resample(time_res).interpolate(method='linear')[:-1]).T
	heights = np.linspace(station.altitude + station.start_bin_height,
                      station.altitude + station.end_bin_height,
                      station.n_bins)
	interp_sigma_df.index = heights
	interp_sigma_df.columns.freq = None
	interp_beta_df.index = heights
	interp_beta_df.columns.freq = None

	'''Calculate the molecular attenuated backscatter as :  beta_mol * exp(-2*tau_mol)'''
	e_tau_df = interp_sigma_df.apply(prep.cal_e_tau_df,0,args=(haifa_station.altitude,),result_type='expand')
	print ('e_tau ',e_tau_df)
	att_bsc_mol_df = interp_beta_df.multiply(e_tau_df)


	import xarray as xr
	mol_xr= xr.DataArray(data=[interp_beta_df,interp_sigma_df,att_bsc_mol_df],
                     coords=[['beta','sigma','att_bsc_mol'],interp_beta_df.index,interp_beta_df.columns],
                     dims = ['profiles','height','time'],
                     name='molecular')#,interp_beta_df.index,interp_beta_df.columns )



	return att_bsc_mol_df,mol_xr

In [20]:
print('df_mol')
lambda_um = gs.LAMBDA_um().G
haifa_station = gs.station()
mol_df,mol_xr = generate_daily_att_bsc_mol(lambda_um, haifa_station, time_res='30S')
mol_xr

df_mol


NameError: name 'gdas_dst_paths' is not defined

In [ ]:
cur_df= mol_df
x_lims = mdates.date2num([cur_df.columns[0],cur_df.columns[-1]])
y_lims = [cur_df.index[0]*1E-3, cur_df.index[-1]*1E-3]
extent = [x_lims[0],x_lims[1],y_lims[0],y_lims[1] ]
fig, ax = plt.subplots(figsize=(16,15))
im = ax.imshow(cur_df, origin='lower',aspect='auto',cmap = 'turbo',
               extent=extent)#[0,2779,interp_sigma_df.index[0], interp_sigma_df.index[-1]])

ax.set_ylabel('Altitude [km]',fontsize = 24)
ax.set_xlabel('Time',fontsize = 24)
plt.rc('xtick',labelsize = 18)
plt.rc('ytick',labelsize = 18)
fig.colorbar(im,ax= ax)
ax.xaxis_date()
ax.get_xaxis().set_major_locator(mdates.MinuteLocator(interval=120))
xfmt = mdates.DateFormatter('%H:%M')
ax.xaxis.set_major_formatter(xfmt)
fig.tight_layout()
plt.show()

In [ ]:
# TODO: generate daily molecular mol_df - for several days
# TODO: validate units of height, speed of light, sigma and beta !!! (km or m)
# TODO: define attributes of mol_xr
# TODO: create xarray to a daily molecular profile for three channels (UV,G,IR)
# TODO: convert from xarray to pytorch
# TODO: save xarray as netcd file
# TODO: create table of profiles (for the data loader)

# TODO: Create samples of 30 min acording to 'profiles' times
# for the followings :
# X = {lidar measurement (range corrected) molecular (range corrected)}
# Y = {lidar const, reference height [min,max]}

## AERONET : https://aeronet.gsfc.nasa.gov/cgi-bin/data_display_aod_v3?site=Technion_Haifa_IL&nachal=0&year=2017&month=5&day=19&aero_water=0&level=3&if_day=0&if_err=0&place_code=10&DATA_TYPE=-999&year_or_month=3
## it is possible to merge with Terra MODIS or Aqua MODIS -
# / TODO: locate the function that does donwload of sunphotometer data to cameranetwork (maybe Shubi knows this)
# / TODO: ask about the relevant product from MODIS to our porpose.

